In [1]:
def user_books_from_goodreads(user_code = None, shelf_name = None, verbose = 1):
    """
    Scarping book info from Goodreads Users's page
    the user_code is 9 digit code found in the link of user's 'My Books' page
    If no shelf name is provided, info from 'All' shelf is returned
    verbose has usual meaning with value being integer 0 or 1, default is 1.
    eg. book_from_goodreads(user_code =110673662, shelf_name = 'currently-reading', verbose = 1)
    """

    #Scarping book info from Goodreads

    #Imports
    import requests
    import bs4
    import pandas as pd

    #Containers
    books = []
    authors = []
    isbn13s = []
    isbns =[]
    pagess = []
    ratings = []
    num_ratings = []
    pub_dates = []
    date_started = []
    date_completed = []

    p=1
    while True:

        if not shelf_name:
            link = f'https://www.goodreads.com/review/list/{user_code}?page={p}'
        else:
            link = f'https://www.goodreads.com/review/list/{user_code}?page={p}&ref=nav_mybooks&shelf={shelf_name}'

        code = requests.get(link)
        soup = bs4.BeautifulSoup(code.text, "lxml")

        #Breaking the loop
        if soup.select('.bookalike') == []:
            break

        if verbose == 0:
            pass 
        else:
            print(f'Currently on Page: {p}')

        for i in range(len(soup.select('.bookalike'))):

            book_name = soup.select('.bookalike .title a')[i]['title']
            books.append(book_name)

            author = soup.select('.bookalike .author a')[i].text
            authors.append(author)

            isbn13 = soup.select('.bookalike .isbn13 .value')[i].text.strip()
            isbn13s.append(str(isbn13))
            isbn = soup.select('.bookalike .isbn .value')[i].text.strip()
            isbns.append(str(isbn))

            pages = soup.select('.bookalike .num_pages .value')[i].text.split('\n')[1].strip()
            pagess.append(pages)

            rating = float(soup.select('.bookalike .avg_rating .value')[i].text.strip())
            ratings.append(rating)

            num = soup.select('.bookalike .num_ratings .value')[i].text.strip()
            num_rating = int(''.join(num.split(',')))
            num_ratings.append(num_rating)

            pub_date = soup.select('.bookalike .date_pub .value')[i].text.strip()
            pub_dates.append(pub_date)
            
            started = soup.select('.bookalike .date_started')[i].text.split('\n')[3]
            date_started.append(started)

            ended = soup.select('.bookalike .date_read')[i].text.split('\n')[3]
            date_completed.append(ended)

        #Next page
        p=p+1

    #Creating the DataFrame
    index = 'Book Name, Author, ISBN 13, ISBN, Number of Pages, Average Rating, Total Number of Ratings, Publication Date, Date Started, Date Completed'.split(', ')
    book_df = pd.DataFrame(data = [books, authors, isbn13s, isbns, pagess, ratings, num_ratings, pub_dates, date_started, date_completed], index = index).transpose()
    return book_df

In [4]:
df = user_books_from_goodreads(user_code = 110673662, verbose = 1)

Currently on Page: 1
Currently on Page: 2
Currently on Page: 3
Currently on Page: 4
Currently on Page: 5
Currently on Page: 6
Currently on Page: 7
Currently on Page: 8
Currently on Page: 9
Currently on Page: 10
Currently on Page: 11
Currently on Page: 12
Currently on Page: 13
Currently on Page: 14
Currently on Page: 15


In [5]:
df.to_csv(r'C:\Users\harik\Desktop\Book-Gallery\Python\books_for_home.csv', index = False)

In [7]:
df.head()

,Book Name,Author,ISBN 13,ISBN,Number of Pages,Average Rating,Total Number of Ratings,Publication Date,Date Started,Date Completed
0,The Lightning Tree,"Rothfuss, Patrick",,,93,4,2156,2014,"Apr 16, 2022","Apr 16, 2022"
1,What If? Serious Scientific Answers to Absurd ...,"Munroe, Randall",9780544272996,0544272994,303,4.14,151303,"Sep 02, 2014",not set,not set
2,"Leviathan Falls (The Expanse, #9)","Corey, James S.A.",9780356510392,0356510395,528,4.54,31066,"Nov 30, 2021",not set,not set
3,"Tiamat's Wrath (The Expanse, #8)","Corey, James S.A.",9780316332873,0316332879,534,4.57,47842,"Mar 26, 2019",not set,not set
4,"Persepolis Rising (The Expanse, #7)","Corey, James S.A.",9780316332828,0316332828,608,4.37,53386,"Dec 05, 2017","Jun 02, 2022",not set


In [1]:
def goodreads_author(author_id = None, author_name = None, verbose = 1, clean = True):
    
    """
    Works only for Goodread Authors
    needs author_id and author_name
    author_name should be specifically defined or pass clean as False
    """

    import re
    import pandas as pd
    import numpy as np

    names = []
    authors = []
    ratings = []
    published_years = []
    num_ratings = []

    flag = True
    p=1
    
    if clean and author_name:
        return None
    elif clean:
        print('You should provide the author name or use \"clean\" = False')
        return None
    else:
        return None
    
    while flag:
        print(p)
        try:
            link = f"https://www.goodreads.com/author/list/{author_id}?page={p}"

            df = pd.read_html(link)[0]
            for i in range(len(df[1])):
                book_name = df[1].iloc[i].split('by')[0].strip()
                other = df[1].iloc[i].split('by')[1]

                names.append(book_name)

                author = other.split('(')[0].strip()
                authors.append(author)
                text = other.split('(')[-1]
                pattern_rating = r"\d\.\d\d"
                pattern_num = r"\d{1,}\,\d{1,}"
                pattern_year = r"\d{4}"

                if re.findall(pattern_rating, text):
                    ratings.append(re.findall(pattern_rating, text)[0])
                else:
                    ratings.append(np.nan)


                if re.findall(pattern_year, text):
                    published_years.append(re.findall(pattern_year, text)[0])
                else:
                    published_years.append(np.nan)

                if re.findall(pattern_num, text):
                    num_ratings.append(re.findall(pattern_num, text)[0])
                else:
                    num_ratings.append(np.nan)
        except :
            break
        p = p+1

    index = 'Book Name, Author, Average Rating, Publication Date, Total Number of Ratings'.split(', ')
    book_df = pd.DataFrame(data = [names, authors, ratings, published_years, num_ratings], index = index).transpose()
    if clean and author_name:
        book_df=book_df.dropna(thresh=3)
        book_df[book_df['Author'].apply(lambda x: author_name in x)]
        return book_df
    else:
        return book_df

In [253]:
text = string[0].split(' by ')[1]

In [254]:
text

'Jane Austen  4.27 avg rating — 3,266,966 ratings  —  published  1813  —  11651 editions'

In [256]:
author = re.findall(pattern, text)[0]

In [262]:
pattern_rating = r"\d\.\d\d"
pattern_num1 = r"\d\,\d{1,}\,\d{1,}"
pattern_num2 = r"\d{1,}\,\d{1,}"
pattern_year = r"\d{4}"

In [265]:
re.findall(pattern_num1, text)

['3,266,966']

In [268]:
re.findall(pattern_year, text)[0]

'1813'

In [3]:
def author(author_id = None, author_name = None, verbose = 1, clean = True):
    
    """
    Works for any Author on Goodreads
    needs author_id and author_name
    author_name should be specifically defined or pass clean as False
    """

    import re
    import pandas as pd
    import numpy as np

    names = []
    authors = []
    ratings = []
    published_years = []
    num_ratings = []

    flag = True
    p=1
    
    # if clean and author_name:
    #     return None
    if clean:
        print('You should provide the author name or use \"clean\" = False')
        return None
    # else:
    #     return None
    
    while flag:
        print(f"Currently on: {p}")
        try:
            link = f"https://www.goodreads.com/author/list/{author_id}?page={p}"

            df = pd.read_html(link)
            for i in range(len(df[1])):
                string = df[0][1]
                book_name = string[0].split(' by ')[0].strip()
                names.append(book_name)
                
                text = string[0].split(' by ')[1]

                pattern_rating = r"\d\.\d\d"
                pattern_num1 = r"\d\,\d{1,}\,\d{1,}"
                pattern_num2 = r"\d{1,}\,\d{1,}"
                pattern_year = r"\d{4}"
                pattern_author = author_name
                
                author = re.findall(pattern_author, text)[0]
                authors.append(author)

                if re.findall(pattern_rating, text):
                    ratings.append(re.findall(pattern_rating, text)[0])
                else:
                    ratings.append(np.nan)


                if re.findall(pattern_year, text):
                    published_years.append(re.findall(pattern_year, text)[0])
                else:
                    published_years.append(np.nan)

                if re.findall(pattern_num1, text):
                    num_ratings.append(re.findall(pattern_num1, text)[0])
                elif re.findall(pattern_num2, text):
                    num_ratings.append(re.findall(pattern_num2, text)[0])
                else:
                    num_ratings.append(np.nan)
        except:
            pass
        p = p+1

    index = 'Book Name, Author, Average Rating, Publication Date, Total Number of Ratings'.split(', ')
    book_df = pd.DataFrame(data = [names, authors, ratings, published_years, num_ratings], index = index).transpose()
    if clean and author_name:
        book_df=book_df.dropna(thresh=3)
        book_df[book_df['Author'].apply(lambda x: author_name in x)]
        return book_df
    else:
        return book_df

In [4]:
author(author_id=239579, clean=False, author_name='Charles Dickens')

Currently on: 1
Currently on: 2
Currently on: 3
Currently on: 4
Currently on: 5
Currently on: 6
Currently on: 7
Currently on: 8
Currently on: 9
Currently on: 10
Currently on: 11
Currently on: 12
Currently on: 13
Currently on: 14
Currently on: 15
Currently on: 16
Currently on: 17
Currently on: 18
Currently on: 19
Currently on: 20
Currently on: 21
Currently on: 22
Currently on: 23
Currently on: 24
Currently on: 25
Currently on: 26


In [9]:
goodreads_author(author_id=1265, author_name='Terry Goodkind')